# PX912: Solid Mechanics

## Workshop 4

Last week, we considered a simple finite element problem in 1D. This week, instead we will look at a 2D problem of load applied to a trapezoidal panel fixed to a wall. I have tried to break the problem down into smaller steps to help you understand the process.

### Please run the cell below!
This cell loads the core library written for this module. The core library contains hints, solution checking and grading. 

Make sure that the output of the previous cell is $\texttt{Library Loaded!}$. 

In [ ]:
import sys, os
sys.path.insert(0, os.getcwd()+'grader')
from grader import workshop4 as ws4
from grader import practice

grader = ws4.Workshop4()

Imports as usual:

In [ ]:
# SymPy Library: Symbolic Python
import sympy as sym

# Tell sympy to print things nicely
sym.init_printing()

# Matplotlib for plotting
import matplotlib.pyplot as plt

# Numpy for numerics
import numpy as np

## Question 1
Note that in the lecture notes, definitions for $\mathbf{K}^e$ and $\mathbf{f}^e$ require integration. For most problems, this integration process must be carried out numerically. Here we'll use Gauss quadrature in order to do this.

Implement Gauss quadrature to obtain exact values for the following
integrals, and verify by analytical integration.

**a)** 
$$
I_a = \int^{1}_{-1} \left( \xi^4 + 2 \xi^2 \right) \text{d}\xi
$$

In [ ]:
numerical_a = ...
    
print('Numerical result is: ', numerical_a)

exact_a = ...

print('Exact result is: ', exact_a)

In [ ]:
# HINT AND SOLUTION
# grader.hint1a()
grader.check1a(numerical_a, exact_a)

**b)** 
$$
I_b = \int^{4}_{0} \left( x^2 + 1 \right) \text{d}x
$$

In [ ]:
numerical_b = ...

print('Numerical result is: ', numerical_b)

exact_b = ...

print('Exact result is: ', exact_b)

In [ ]:
# HINT AND SOLUTION
# grader.hint1b()
grader.check1b(numerical_b, exact_b)

## Question 2
We consider a linear elasticity problem on a trapezoidal panel domain, as shown below:

<img src="grader/pictures/workshop4.png" alt="Drawing" style="width: 800px;"/>

The panel is made of a material with Young’s modulus
$E =30$ MPa and Poisson’s ratio $\nu=0.3$. Plane stress conditions are
assumed. The problem is discretised with a linear quadrilateral element.
Calculate strains and stresses at element integration (Gauss) points using
the finite-element approach.

### Inputs

First, specify the elasticity matrix for this problem, using the notes to help.

In [ ]:
def Plane_Stress_C(E, nu):
    ...

In [ ]:
E = 30e6
nu = 0.3

ps = Plane_Stress_C(E, nu)
print(ps)

In [ ]:
# HINT AND SOLUTION
# grader.hint2ai()
grader.check2ai(ps)

Identify the **fixed displacements** within the global displacement vector. Which indices of the vector below will have zero value?

$$
\mathbf{d}^e = [d^e_{1(1)}, d^e_{2(1)}, d^e_{1(2)},d^e_{2(2)}, d^e_{1(3)}, d^e_{2(3)}, d^e_{1(4)}, d^e_{2(4)}] 
$$

In [ ]:
# YOUR CODE HERE
fixed_disp = ...

In [ ]:
# HINT AND SOLUTION
# grader.hint2aii()
grader.check2aii(fixed_disp)

We will also need the nodal force vector, which takes the form.

$$
\mathbf{f}^e = [f^e_{1(1)}, f^e_{2(1)}, f^e_{1(2)},f^e_{2(2)}, f^e_{1(3)}, f^e_{2(3)}, f^e_{1(4)}, f^e_{2(4)}] 
$$
This we will specify later.

Let's get the coordinate matrix, or the mapping between the normalised coordinates and the real coordinates. When running the real calculation, you probably don't need a function to automatically translate points. However, you need the mapping function to calculate important quantities - it is always useful to put it in function form.

In [ ]:
def xf(xi, eta):
    ...

def yf(xi, eta):
    ...

In [ ]:
print(xf(-1,-1), yf(-1,-1))
print(xf(1,-1), yf(1,-1))
print(xf(-1,1), yf(-1,1))
print(xf(1,1), yf(1,1))

In [ ]:
# HINT AND SOLUTION
# grader.hint2aiii()
grader.check2aiii(xf, yf)

Now let's specify the shape functions. There should be one for each node

In [ ]:
# YOUR CODE HERE
# you might like to write a function that calculates all of these at once
N_1 = 0
N_2 = 0
N_3 = 0
N_4 = 0

In [ ]:
# HINT
# grader.hint2aiv()
grader.check2aiv(N)

### Finite Element Matrices

We should now specify the strain-displacement matrix

$$
\mathbf{B}^e = [\mathbf{B}^e_1, \mathbf{B}^e_2, \mathbf{B}^e_3, \mathbf{B}^e_4]
$$

In [ ]:
def Disp_Strain_B(real_coords, xi, eta):
    ...

In [ ]:
# HINT
# grader.hint2bi()

And the Jacobian:

$$
\mathbf{J}^e =
\begin{bmatrix}
\frac{\partial x_1}{\partial \xi} & \frac{\partial x_2}{\partial \xi} \\
\frac{\partial x_1}{\partial \eta} & \frac{\partial x_2}{\partial \eta}
\end{bmatrix}
$$

In [ ]:
def J(xi, eta):
    ...

def J_inv(xi, eta):
    ...

In [ ]:
# HINT
# grader.hint2bii()

Then at each integration point we need to build the stiffness matrix from the Jacobian, the strain-displacement matrix, and the elasticity matrix:

$$
\mathbf{K}^e(\xi_i, \eta_i) = {\mathbf{B}^e}^T(\xi_i, \eta_i)) \mathbf{C} \mathbf{B}^e(\xi_i, \eta_i) \det(\mathbf{J}^e(\xi_i, \eta_i)
$$

In [ ]:
def keval(real_coords, Ce):
    ...

In [ ]:
# HINT
# grader.hint2biii()

These should then be assembled into the elemental stiffness matrix:

$$
\mathbf{K}^e = \sum_i \sum_j \mathbf{K}(\xi_i, \eta_j)
$$

In [ ]:
# YOUR CODE HERE
#Boundary conditions
coords = ...

# Calculate element stiffness matrix
C = ...
ke = ...

# Remove components of the stiffness matrix that correspond to zero DOFs

print(ke)

In [ ]:
# HINT
grader.check2biii(ke)

Almost there! Now we add the external loads into the nodal force vector:

$$
\mathbf{f}^e = \int_{\Gamma_{14}}{\mathbf{N}^e}^T \bar{\mathbf{t}} \, \text{d}\Gamma
$$

In [ ]:
# Applied loads - define the RHS load vector 
fext_top = ...

# create RHS vector
f_e = ...

In [ ]:
# HINT
# grader.hint2biv()
grader.check2biv(f_e)

### Solution

Now we need to reduce the overall system of equations and solve for the displacements.

In [ ]:
# Calculate non-zero nodal displacements
ue = ...

# Create the global nodal displacement vector 
d_e = ...
print(d_e)

In [ ]:
# HINT
# grader.hint2ci()
grader.check2ci(d_e)

In [ ]:
# Plot element deformation (nodal displacements)
sdef=10000

plt.plot(coords[:, 0],
         coords[:, 1], 'sk')
plt.plot(coords[:, 0] + sdef*d_e[0:2*len(coords)-1:2],
         coords[:, 1] + sdef*d_e[1:2*len(coords):2], 'or')

for i in range(len(coords)):
    plt.fill(coords[:, 0],
             coords[:, 1], edgecolor='k', fill=False)
    plt.fill(coords[:, 0] + sdef*d_e[0:2*len(coords)-1:2],
             coords[:, 1] + sdef*d_e[1:2*len(coords):2], edgecolor='r', fill=False)
plt.show()

We can then solve for the strains at the four Gauss points:
$$
\varepsilon^e(\xi_i, \eta_j) = \mathbf{B}^e(\xi_i, \eta_j) \mathbf{d}^e = [\varepsilon^e_{11}, \varepsilon^e_{22}, \gamma^e_{12}]^T
$$

In [ ]:
# YOUR CODE HERE

def strains(real_coords, de):
    ...
        
strain_vals = ...
print(strain_vals)

In [ ]:
# HINT
# grader.hint2cii()
grader.check2cii(strain_vals)

And similarly the stresses:

$$
\sigma^e(\xi_i, \eta_j) = \mathbf{C}\varepsilon^e(\xi_i, \eta_j) = [\sigma^e_{11}, \sigma^e_{22}, \sigma^e_{12}]^T
$$

Return the *sum* of stresses over the Gauss points.

In [ ]:
def sigma_vector(coords,C,de):
    ...

sigma = sigma_vector(coords,C,d_e)

print(sigma)

In [ ]:
# HINT
# grader.hint2ciii()
grader.check2ciii(sigma)

And that completes the solution to the FE problem!

# Results

Run the box below to check your progress.

In [ ]:
grader.results()